# SQS

This notebook will walk you through how to use Boto3 with an AWS service. SQS allows you to queue and then process messages. This notebook focusses on Amazon Simple Queue Service (SQS), how to create a new queue, get and use an existing queue, push new messages onto the queue and process messages from the queue by using Resources and Collections.

### Creating a Queue

Queues when created will be given a name. Optional queue attributes can be set, such as the number of seconds to wait before an item may be processed. Before creating a queue, you must first get the SQS service resource:

In [ ]:
# We have already configured aws credentials using aws configure command in AWS CLI. So we dont have to provide 
# credentials.

import boto3

[Reference for AWS Configuration in local machine](http://boto3.readthedocs.io/en/latest/guide/quickstart.html#configuration) 

In [ ]:
# Get the service resource
sqs = boto3.resource('sqs')


# Create the queue. This returns an SQS.Queue instance, modify with your pawprint
queue = sqs.create_queue(QueueName='test_queue_your_pawprint', Attributes={'DelaySeconds': '5'})


# You can now access identifiers and attributes
print(queue.url)
print(queue.attributes.get('DelaySeconds'))

<a id="setting_queue_variable"></a>


### Using an Existing Queue

You can look up for a queue by its name. If the queue does not exist, then an exception will be thrown.

In [ ]:
# Get the queue. This returns an SQS.Queue instance
queue = sqs.get_queue_by_name(QueueName='test_queue_your_pawprint')

# You can now access identifiers and attributes
print(queue.url)
print(queue.attributes.get('DelaySeconds'))


To list all the existing queues.

In [ ]:
# Print out each queue name, which is part of its ARN
for queue in sqs.queues.all():
    print(queue.url)

**Note: ** To extract the names of queues, you can use queue's Amazon Resource Names (ARNs). ARNs are available in the queue's attributes. For example, 

[Amazon Resource Names](http://docs.aws.amazon.com/general/latest/gr/aws-arns-and-namespaces.html)

Amazon Resource Names (ARNs) are used to uniquely identify AWS resources. We require an ARN when you need to specify a resource unambiguously across all of AWS, such as in IAM policies, Amazon Relational Database Service (Amazon RDS) tags, and API calls.

In [ ]:
for queue in sqs.queues.all():
    print(queue.attributes['QueueArn'].split(':')[-1])

### Sending Messages

Sending a message adds it to the end of the queue. So queue is pointing to test_queue right now as we have it [set above](#setting_queue_variable)

In [ ]:
# Get the queue. This returns an SQS.Queue instance
queue = sqs.get_queue_by_name(QueueName='test_queue')

# Create a new message
response = queue.send_message(MessageBody='DSA Fall sesmester', 
                              MessageAttributes={'Term': {'StringValue': 'Fall 2017',
                                                          'DataType': 'String'
                                                         }
                                                 'Course': {'StringValue': 'Main',
                                                          'DataType': 'String'
                                                           }
    
                                                }
                             )

# The response is NOT a resource, but gives you a message ID and MD5
print(response.get('MessageId'))
print(response.get('MD5OfMessageBody'))

You can create messages with custom attributes as shown below.

In [ ]:
queue.send_message(MessageBody='DSA course 1', MessageAttributes={
    'Course': {
        'StringValue': 'Intro_to_Data_Science',
        'DataType': 'String'
    }
    
})

Messages can also be sent in batches. For example, sending the two messages described above in a single request would look like the following:

In [ ]:
response = queue.send_messages(Entries=[
    {
        'Id': '1',
        'MessageBody': 'DSA course 2',
        'MessageAttributes': {
            'Course': {
                'StringValue': 'Statistical & mathematical foundations of Data Analysis',
                'DataType': 'String'
            }
        }
    },
    {
        'Id': '2',
        'MessageBody': 'DSA course 3',
        'MessageAttributes': {
            'Course': {
                'StringValue': 'Database Analytics',
                'DataType': 'String'
            }
        }
    }
])

# Print out any failures
print(response.get('Failed'))


# Response contains list of Successful and Failed messages, so you can retry failures if needed.

### Processing Messages

Messages are processed in batches.

In [ ]:
# Process messages by printing out body and optional author name
for message in queue.receive_messages(MessageAttributeNames=['Course']):
    # Get the custom course message attribute if it was set
    course_text = ''
    if message.message_attributes is not None:
        course_name = message.message_attributes.get('Course').get('StringValue')
        
    # Print out the body and author (if set)
    print('Hello, {0}!{1}'.format(message.body, course_text))

    # Let the queue know that the message is processed
#     message.delete()

# Save your Notebook, then `File > Close and Halt`